### Multi-agent supervisor

In this project we'll be building a supervisor system with two agents - a reasearch and a math expert.

We'll - 
- Build specialized research and math agents
- Build a supervisor for orchestrating them with the prebuilt langgraph-supervisor
- Build a supervisor from scratch
- Implement advanced task delegation

In [4]:
# %pip install -U langgraph langgraph-supervisor tavily-python langchain-google-genai

In [2]:
import getpass
import os

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = input(f"{var}: ")

_set_env("GEMINI_API_KEY")
_set_env("TAVILY_API_KEY")

### 1. Creating Worker agents

We'll now create our agents - research agent and math agent

- Research agent will have access to a web search tool using the tavily api
- Math agent will have access to simple math tools

In [14]:
from tavily import TavilyClient
from langchain.tools import tool

tavily_client = TavilyClient(api_key = os.getenv("TAVILY_API_KEY"))

# web_search = tavily_client.search(query = "Who has the most ballon dors", max_results=3)

# print(web_search["results"][0]["content"])

@tool("web_search")
def web_search(query: str):
    """
    Search the web using Tavily
    """
    result = tavily_client.search(query=query, max_results=3)
    return result

In [15]:
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    google_api_key = os.getenv("GEMINI_API_KEY"),
    temperature = 0
)

research_agent = create_agent(
    model,
    tools= [web_search],
    system_prompt=(
        "You are a research agent.\n\n"
        "INSTRUCTIONS:\n"
        "- Assist ONLY with research-related tasks, DO NOT do any math\n"
        "- After you're done with your tasks, respond to the supervisor directly\n"
        "- Respond ONLY with the results of your work, do NOT include ANY other text."
    ),
    name="research_agent"
)

In [23]:
from langchain_core.messages import convert_to_messages


def pretty_print_message(message, indent=False):
    content = message.content

    if isinstance(content, list) and len(content) > 0:
        # Gemini structured response format
        if isinstance(content[0], dict) and "text" in content[0]:
            content = content[0]["text"]

    pretty_message = message.pretty_repr(html=True).replace(str(message.content), str(content))

    if not indent:
        print(pretty_message)
        return

    indented = "\n".join("\t" + c for c in pretty_message.split("\n"))
    print(indented)



def pretty_print_messages(update, last_message=False):
    is_subgraph = False
    if isinstance(update, tuple):
        ns, update = update
        # skip parent graph updates in the printouts
        if len(ns) == 0:
            return

        graph_id = ns[-1].split(":")[0]
        print(f"Update from subgraph {graph_id}:")
        print("\n")
        is_subgraph = True

    for node_name, node_update in update.items():
        update_label = f"Update from node {node_name}:"
        if is_subgraph:
            update_label = "\t" + update_label

        print(update_label)
        print("\n")

        messages = convert_to_messages(node_update["messages"])
        if last_message:
            messages = messages[-1:]

        for m in messages:
            pretty_print_message(m, indent=is_subgraph)
        print("\n")

for chunk in research_agent.stream(
    {"messages": [{"role": "user", "content": "who is the mayor of NYC?"}]}
):
    pretty_print_messages(chunk)

Update from node model:


================================== Ai Message ==================================
Tool Calls:
  web_search (c998a686-3cd8-4aec-9299-f29293935261)
 Call ID: c998a686-3cd8-4aec-9299-f29293935261
  Args:
    query: mayor of NYC


Update from node tools:


================================= Tool Message =================================
Name: web_search

{"query": "mayor of NYC", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.facebook.com/NYCMayor/", "title": "Mayor Eric Adams (@NYCMayor) - Facebook", "content": "Mayor Eric Adams. 406225 likes · 12541 talking about this. Official account of the 110th Mayor of NYC.", "score": 0.8033131, "raw_content": null}, {"url": "https://www.instagram.com/nycmayor/?hl=en", "title": "Mayor Eric Adams (@nycmayor) · New York, NY - Instagram", "content": "983K followers · 4.6K+ posts · Delivering a fairer, safer, and brighter future for Every New Yorker. Official account of the 110th Mayor

In [24]:
@tool("add")
def add(a: float, b: float):
    """Add two numbers."""
    return a + b


@tool("multiply")
def multiply(a: float, b: float):
    """Multiply two numbers."""
    return a * b


@tool("divide")
def divide(a: float, b: float):
    """Divide two numbers."""
    return a / b

math_agent = create_agent(
    model,
    tools = [add, multiply, divide],
    system_prompt=(
        "You are a math agent.\n\n"
        "INSTRUCTIONS:\n"
        "- Assist ONLY with math-related tasks\n"
        "- After you're done with your tasks, respond to the supervisor directly\n"
        "- Respond ONLY with the results of your work, do NOT include ANY other text."
    ),
    name = "math_agent"
)

In [25]:
for chunk in math_agent.stream(
    {"messages": [{"role": "user", "content": "what's (3 + 5) x 7"}]}
):
    pretty_print_messages(chunk)

Update from node model:


================================== Ai Message ==================================
Tool Calls:
  add (38261866-7d64-4887-abcb-8aa32893e0dd)
 Call ID: 38261866-7d64-4887-abcb-8aa32893e0dd
  Args:
    a: 3
    b: 5


Update from node tools:


================================= Tool Message =================================
Name: add

8.0


Update from node model:


================================== Ai Message ==================================
Tool Calls:
  multiply (8696c462-3e06-48d5-b2da-001dbb316e40)
 Call ID: 8696c462-3e06-48d5-b2da-001dbb316e40
  Args:
    a: 8
    b: 7


Update from node tools:


================================= Tool Message =================================
Name: multiply

56.0


Update from node model:


================================== Ai Message ==================================

56


